In [402]:
# all imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

# import data and shuffle it
data = pd.read_csv('../data/Pima_Indian_diabetes.csv');
data = data.sample(frac=1, random_state=15).reset_index(drop=True)
data_shape = data.shape
data_columns = data.columns

In [403]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,742.000000,752.000000,768.000000,746.000000,768.000000,757.000000,768.000000,749.000000,768.000000
mean,3.866601,119.966097,68.886078,20.309879,79.799479,31.711151,0.471876,33.761336,0.348958
std,3.479971,32.367659,19.427448,15.974523,115.244002,8.544789,0.331329,12.297409,0.476951
min,-5.412815,0.000000,-3.496455,-11.945520,0.000000,-16.288921,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.100000,0.243750,24.000000,0.000000
50%,3.000000,116.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.250000,36.500000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [404]:
data_neg_changed = data.assign(
                               Pregnancies = lambda x: x.Pregnancies.where(x.Pregnancies.ge(0)),
                               Glucose = lambda x: x.Glucose.where(x.Glucose.gt(0)),
                               BloodPressure = lambda x: x.BloodPressure.where(x.BloodPressure.gt(0)),
                               SkinThickness = lambda x: x.SkinThickness.where(x.SkinThickness.gt(0)),
                               Insulin = lambda x: x.Insulin.where(x.Insulin.gt(0)),
                               BMI = lambda x: x.BMI.where(x.BMI.gt(0))
                              )

In [405]:
print(data_neg_changed.shape)
print(data_neg_changed.isnull().sum())

(768, 9)
Pregnancies                  29
Glucose                      21
BloodPressure                35
SkinThickness               242
Insulin                     374
BMI                          25
DiabetesPedigreeFunction      0
Age                          19
Outcome                       0
dtype: int64


In [406]:
data_neg_changed.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,739.000000,747.000000,733.000000,526.000000,394.000000,743.000000,768.000000,749.000000,768.000000
mean,3.902135,120.769083,72.185074,28.874152,155.548223,32.359164,0.471876,33.761336,0.348958
std,3.441810,30.944811,12.502210,10.598375,118.775855,7.152753,0.331329,12.297409,0.476951
min,0.000000,42.974768,15.372031,7.000000,14.000000,5.317899,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,21.000000,76.250000,27.450000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,140.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [407]:
data_neg_changed.drop(['SkinThickness', 'Insulin'], axis=1, inplace=True)
data_neg_changed.shape

(768, 7)

In [408]:
data_neg_changed.dropna(axis=0, how='any', inplace=True)
data_neg_changed.shape

(650, 7)

In [409]:
data_neg_changed.describe()

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age,Outcome
count,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000
mean,3.910291,120.894412,72.120981,32.386063,0.477469,33.632595,0.341538
std,3.439186,30.837428,12.290524,7.054841,0.341095,11.821947,0.474591
min,0.000000,42.974768,15.372031,6.699051,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,27.500000,0.245000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,32.400000,0.377000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,36.600000,0.628500,41.000000,1.000000
max,17.000000,199.000000,110.000000,67.100000,2.420000,69.312541,1.000000


In [410]:
data_neg_changed['Outcome'].value_counts()

0    428
1    222
Name: Outcome, dtype: int64

In [411]:
X = data_neg_changed.iloc[:, :-1]
y = data_neg_changed['Outcome']

In [412]:
feature_cols = X.columns
mm_scaler = MinMaxScaler()
X = mm_scaler.fit_transform(X)
X = pd.DataFrame(X, columns=feature_cols)

In [413]:
X.describe()

,Pregnancies,Glucose,BloodPressure,BMI,DiabetesPedigreeFunction,Age
count,650.000000,650.000000,650.000000,650.000000,650.000000,650.000000
mean,0.230017,0.499404,0.599706,0.425275,0.170568,0.261477
std,0.202305,0.197644,0.129883,0.116800,0.145643,0.244697
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.058824,0.359078,0.513886,0.344381,0.071307,0.062096
50%,0.176471,0.474444,0.598427,0.425506,0.127669,0.165588
75%,0.352941,0.621856,0.682969,0.495041,0.235056,0.413971
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [414]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=10)

In [415]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param = {'C':[1,2,3,4,5,6,7,8,9,10]}
lrcv = LogisticRegression()
clf = GridSearchCV(lrcv, param, cv=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7384615384615385

In [416]:
from sklearn import svm, datasets
# from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
param = {'cv':[1,2,3,4,5], 'Cs':[1,2,3,4,5,6,7,8,9,10]}
lrcv = LogisticRegressionCV()
clf = GridSearchCV(lrcv, param, cv=5)
clf.fit(iris.data, iris.target)

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
# # replacing with median of that column
# for col in data.columns:
#     if(col != 'Outcome'):
#         data[col].fillna(data[col].median(), inplace=True)

In [ ]:
# x = data.iloc[:, :-1]
# Y = data['Outcome']

In [ ]:
# # ensuring that datatype of all columns is float64
# x.dtypes

In [ ]:
# # scale the feature columns
# cols = x.columns
# # using minmax scaler as units of feature columns are different and this brings them between 0 and 1
# # mm_scaler = MinMaxScaler() 
# std_scaler = StandardScaler()
# x = std_scaler.fit_transform(x)
# x = pd.DataFrame(x, columns=cols)
# x.describe()

In [ ]:
# # split training and testing data
# X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.3, random_state=15, shuffle=True)

In [ ]:
# y_train.value_counts()

In [ ]:
# # logistic regression
# clf = LogisticRegressionCV(cv=5, random_state=15).fit(X_train, y_train)
# clf.score(X_test, y_test)

# # SVM
# # clf = SVC()
# # clf.fit(X_train, y_train)
# # clf.score(X_test, y_test)

# # SGDClassifier
# # clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
# # clf.fit(X_train, y_train)
# # clf.score(X_test, y_test)

# # KNN classifier
# # neigh = KNeighborsClassifier(n_neighbors=3)
# # neigh.fit(X_train, y_train)
# # neigh.score(X_test, y_test)